In [4]:
import preprocessor as p
import numpy as np
import pandas as pd
import re
import string
import nltk

In [6]:
path='../collected_data/'

In [57]:
df = pd.read_csv('bias_data.csv')
df.head()

,label,text
0,0.0,i'm only currently using freebsd on aws. :p n...
1,0.0,i've posted a contact form at for support reg...
2,0.0,nope. i had to manually block her.
3,0.0,some good stuff here.
4,0.0,wtf ? man gets upset that women in tech are u...


In [58]:
tweets = df.text
tweets.head()

0    i'm only currently using freebsd on aws. :p  n...
1    i've posted a contact form at  for support reg...
2                   nope. i had to manually block her.
3                                some good stuff here.
4    wtf  ? man gets upset that women in tech are u...
Name: text, dtype: object

## clean tweets

In [59]:
punctuation = [':', ';', '!', ',','.']
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [61]:
cleaned_tweets = []
for tweet in tweets:
    tweet = re.sub('\@\w+', '', str(tweet))
    tweet = re.sub('\#\w+', '', tweet)
    tweet = re.sub('\#', '', tweet)
    tweet = re.sub('RT', '', tweet)
    tweet = re.sub('&amp', '', tweet)
    tweet = re.sub('[0-9]+', '', tweet)
    tweet = re.sub('//t.co/\w+', '', tweet)
    tweet = re.sub('w//', '', tweet)
    tweet = tweet.lower()
    cleaned_tweets.append(tweet)

In [62]:
def clean_ascii(text):
    return ''.join(i for i in text if ord(i) < 128)

In [63]:
dataset_1 = []
for idx, row in enumerate(cleaned_tweets):
    new_row = clean_ascii(row)
    ' '.join(new_row.split())
    new_row = new_row.strip()
    dataset_1.append(new_row)

In [65]:
len(dataset_1)

7506

In [73]:
labels = list(df.label)
len(labels)

7506

In [75]:
dataset_2 = np.column_stack((dataset_1, labels))
dataset_2

array([["i'm only currently using freebsd on aws. :p no zfs - not much need for it.",
        '0.0'],
       ["i've posted a contact form at for support regarding , speaking requests, feedback, etc.",
        '0.0'],
       ['nope. i had to manually block her.', '0.0'],
       ...,
       ['update ripped this passive aggressive bullshit the fuck down.',
        '1.0'],
       ['oh hey, statistics', '1.0'],
       ["sometimes i feel i need to remind people that being nostalgic for your childhood isn't universal.",
        '1.0']], dtype='<U140')

In [76]:
dataset = [row for row in dataset_2 if len(row[0].split()) >= 3]

In [77]:
len(dataset)

7329

In [79]:
df_1 = pd.DataFrame(dataset)
df_1.columns=['text', 'label']
df_1.head()

,text,label
0,i'm only currently using freebsd on aws. :p no...,0.0
1,i've posted a contact form at for support rega...,0.0
2,nope. i had to manually block her.,0.0
3,some good stuff here.,0.0
4,wtf ? man gets upset that women in tech are up...,0.0


In [80]:
df_1.to_csv('temp.csv', index=False)

In [81]:
msk = np.random.rand(len(df_1)) < 0.8

In [84]:
train=df_1.sample(frac=0.8,random_state=200)
len(train)

5863

In [85]:
test=df.drop(train.index)

In [86]:
train.to_csv('train_1.csv', index=False)
test.to_csv('test_1.csv', index=False)